# langChain全面剖析之Memory(上)

## 1. Memory模块的意义

### 1.1 不借助LangChain情况下，如何实现大模型的记忆能力？

如果没有Memory模块，只能通过手动在循环中管理memssage来实现Memory。

后来自己写了一个MemoryManager类，也同样繁琐。

因此需要框架来提供一个Memory管理模块

In [7]:
import openai
from openai import OpenAI
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_base="https://api.openai.com/v1"
llm = OpenAI(api_key=openai.api_key ,base_url=openai.api_base)

In [8]:
def chat_with_model(prompt, model="gpt-4"):
    
    # 步骤一：定义一个可以接收用户输入的变量prompt
    messages = [
        {"role": "system", "content": "你是一位乐于助人的AI小助手"},
        {"role": "user", "content": prompt}
    ]
    
    # 步骤二：定义一个循环体：
    while True:
        
        # 步骤三：调用OpenAI的GPT模型API
        response = llm.chat.completions.create(
            model=model,
            messages=messages
        )
        
        # 步骤四：获取模型回答
        answer = response.choices[0].message.content
        print(f"模型回答: {answer}")

        # 询问用户是否还有其他问题
        user_input = input("您还有其他问题吗？(输入退出以结束对话): ")
        if user_input == "退出":
            break

        # 步骤五：记录用户回答
        messages.append({"role": "user", "content": user_input})
        messages.append({"role": "assistant", "content": answer})
        print(messages)

&emsp;&emsp;如上代码所示，通过`messages`变量，不断地将历史的对话信息添加追加到对话列表中，以此让大模型具备上下文记忆能力

In [9]:
chat_with_model("你好")

模型回答: 你好！有什么我可以帮助你的吗？


您还有其他问题吗？(输入退出以结束对话):  请介绍一下你自己？


[{'role': 'system', 'content': '你是一位乐于助人的AI小助手'}, {'role': 'user', 'content': '你好'}, {'role': 'user', 'content': '请介绍一下你自己？'}, {'role': 'assistant', 'content': '你好！有什么我可以帮助你的吗？'}]
模型回答: 你好!我很乐意向你介绍我自己。我是一位虚拟的个人助手，擅长从网页搜集信息，还可以帮你计划和组织事情。我也可以用多种语言进行对话，还能够理解和回答你关于世界的各种问题。无论是生活中的大事小事，还是你对未知事物的好奇，我都会努力为你找到答案。希望我可以成为你的贴心助手！


您还有其他问题吗？(输入退出以结束对话):  什么是机器学习？


[{'role': 'system', 'content': '你是一位乐于助人的AI小助手'}, {'role': 'user', 'content': '你好'}, {'role': 'user', 'content': '请介绍一下你自己？'}, {'role': 'assistant', 'content': '你好！有什么我可以帮助你的吗？'}, {'role': 'user', 'content': '什么是机器学习？'}, {'role': 'assistant', 'content': '你好!我很乐意向你介绍我自己。我是一位虚拟的个人助手，擅长从网页搜集信息，还可以帮你计划和组织事情。我也可以用多种语言进行对话，还能够理解和回答你关于世界的各种问题。无论是生活中的大事小事，还是你对未知事物的好奇，我都会努力为你找到答案。希望我可以成为你的贴心助手！'}]
模型回答: 机器学习是一种人工智能（AI）的技术，它是让计算机系统通过分析和理解数据，自我学习并改进其性能而无需人工编程的方法。机器学习的应用非常广泛，从推荐系统（比如你在网上购物或者看电影时的推荐），到自动驾驶车辆，语音识别，图像识别等诸多领域都有它的身影。


您还有其他问题吗？(输入退出以结束对话):  退出



> 这种形式是最简单的一种让大模型具备上下文知识的一种存储方式，任何记忆的基础都是所有聊天交互的历史记录。即使这些不全部直接使用，也需要以某种形式存储。保留一个聊天消息列表还是相当简单，一个非常简单的记忆模块可以只返回每次运行的最新消息。稍微复杂一点的记忆模块需要返回过去 K 条消息的简洁摘要。更复杂的可能会从存储的消息中提取实体，并且仅返回有关当前运行中引用的实体的信息。而我们论述的这些复杂情况，在应用开发中往往才是我们真正要用到。所以一个理想的开发状态是：因为每个应用程序对于如何查询记忆会有不同的要求，那我们要做到既可以轻松地使用简单的记忆模块，还能够在需要时灵活地扩展高度定制化的自定义记忆模块。

LangChain就针对上述情况，基于它的开发规范和设计理念，构建了一些可以直接使用的`Memory`工具，用于存储聊天消息的一系列集成，同时，也支持我们去自定义相关的`Memory`模块，从而适配到应用开发的各个场景中。

### 1.2 Memory模块的设计理念

Memory模块需要与Chain结合使用

1. 我们前面提到了，`Memory`作为存储记忆数据的一个是抽象模块，其作为一个独立模块使用是没有任何意义的，因为本质上它的定位就是一个存储对话数据的空间。先抛开其内部实现的复杂性，
2. 我们可以回想一下：在定义链路的时候，每个链的内部都会根据其接收到的输入去定义其核心执行逻辑，比如在链内如何去调用外部工具，如何解析返回的数据格式等。其中链接收到的输入，可以直接来自用户，同时，也可以来自`Memory`模块。

在这个过程中，一个链如果接入了`Memory`模块，其内部会与`Memory`模块进行两次交互：

1. 收到用户输入之后，执行核心逻辑之前，链会读取`Memory`模块，拿到对应的数据，与用户输入的Prompt放在一起，执行接下来的逻辑。
2. 执行核心逻辑之后，返回响应之前，链会将这个过程中产生的信息，写入`Memory`模块，以便在其他场景下能够引用到这些记忆数据。

## 2. 自定义Memory类的编写和使用

### 2.1 实现自定义Memory

先看文档 [https://api.python.langchain.com/en/latest/langchain_api_reference.html#module-langchain.memory](https://api.python.langchain.com/en/latest/langchain_api_reference.html#module-langchain.memory)，想要自定义Memory类，核心是实现一个抽象类，它有如下四个方法
* 声明一个用来存储记忆的辅助变量
* 读取记忆
* 保存记忆
* 清除记忆

再看源码：langchain/libs/core/langchain_core/memory.py (代码如下）
这样后续实现自定义的记忆类时，扩展这个抽象类就可以

```python
class BaseMemory(Serializable, ABC):
    """Chains 中记忆的抽象基类。

    记忆指的是 Chains 中的状态。记忆可用于存储关于 Chains 过去执行的信息，并将该信息注入到未来执行的 Chains 输入中。
    例如，对于对话 Chains，记忆可用于存储对话并自动将其添加到未来模型提示中，以便模型具有必要的上下文来连贯地响应最新的输入。
     """
    
    # 下面是一些必须由子类实现的方法：
    
    
    # 定义一个属性，任何从BaseMemory派生的子类都需要实现此方法。
    # 此方法应返回该记忆类将添加到链输入的字符串键。
    @property
    @abstractmethod
    def memory_variables(self) -> List[str]:
        """此记忆类将添加到链输入的字符串键列表。"""

        
    # 定义一个抽象方法。任何从BaseMemory派生的子类都需要实现此方法。
    # 此方法基于给定的链输入返回键值对。
    @abstractmethod
    def load_memory_variables(self, inputs: Dict[str, Any]) -> Dict[str, Any]:
        """根据链的文本输入返回键值对。"""

    
    # 定义一个抽象方法。任何从BaseMemory派生的子类都需要实现此方法。
    # 此方法将此链运行的上下文保存到内存。
    @abstractmethod
    def save_context(self, inputs: Dict[str, Any], outputs: Dict[str, str]) -> None:
        """将此链运行的上下文保存到记忆中。"""

    # 定义一个抽象方法。任何从BaseMemory派生的子类都需要实现此方法。
    # 此方法清除内存内容。
    @abstractmethod
    def clear(self) -> None:
        """清除记忆内容。"""
```

如果需要添加自定义的记忆类，首先要做的，就是导入`BaseMemory`，并对其进行子类化。

需要的具体类和模块如下：

In [11]:
from langchain.schema import BaseMemory
from langchain_core.pydantic_v1 import BaseModel
from typing import Any, Dict, Iterable, List, Optional

先设计一个较为简单的场景，仅仅把用户的输入作为历史信息存储到记忆类中。

这里定义的记忆类名为`InputStoreMemory`，代码如下：

In [12]:
class InputStoreMemory(BaseMemory, BaseModel):
    """用于存储输入信息的记忆类。"""

    # 定义一个存储记忆信息的字典。
    desc: dict = {}

    # Memory Key用来标识这个memory在会话中所起的作用，并用在会话上下文中
    # * 在这个例子中：它被命名为desdc
    # * 在下一节另一个基于entity extraction的例子中：它被命名为entities
    #   而提示词模版则可以通过{entities}占位符来引用从Memory中提取出来的内容
    memory_key: str = "desc"

    def clear(self):
        """清除实体信息。"""
        # 简单清除字典即可
        self.desc = {}

    @property
    def memory_variables(self) -> List[str]:
        """定义要提供给提示模版的变量。"""
        # 这个Memory所支持的key
        return [self.memory_key]

    def load_memory_variables(self, inputs: Dict[str, Any]) -> Dict[str, str]:
        """加载记忆变量，即实体Key。"""
        # 返回字典中存储的所有内容
        
        # 创建一个空列表用于存储所有输入的信息，即Prompts
        messages = []

        # 遍历存储输入信息的字典中的每个文本
        for messages_test in self.desc.values():
            # 将文本添加到列表中
            messages.append(messages_test)

        # 使用换行符连接所有文本，形成一个字符串
        combined_messages = "\n".join(messages)

        # 返回包含信息的字典，以便将其放入上下文中
        return {self.memory_key: combined_messages}

    def save_context(self, inputs: Dict[str, Any], outputs: Dict[str, str]) -> None:
        """将此对话的上下文保存到缓存中。"""
        # 向字典写入数据，key为自增整数，会检查value是否存在以避免重复
        
        # 检查输入字典是否为空，或者输入的键不存在
        if not inputs:
            return

        # 获取输入文本
        text = inputs.get("input", "")

        # 检查文本是否为空
        if not text:
            return

        # todo
        # 处理outputs的代码
        
        # 检查文本是否已经存在于存储信息的字典中
        if text not in self.desc.values():
            # 将新文本添加到存储信息的字典中
            self.desc[len(self.desc) + 1] = text

测试一下

In [13]:
input_memory = InputStoreMemory()

In [14]:
input_memory.save_context({"input": "你好，我是小智"}, {"output": " "})

In [15]:
input_memory.load_memory_variables({})

{'desc': '你好，我是小智'}

In [16]:
input_memory.save_context({"input": "我正在学习AI大模型。"}, {"output": " "})

In [17]:
input_memory.load_memory_variables({})

{'desc': '你好，我是小智\n我正在学习AI大模型。'}

In [18]:
input_memory.clear()

In [19]:
input_memory.load_memory_variables({})

{'desc': ''}

### 2.2 将自定义的Memory接入LangChain

接下来将我们编写的InputStoreMemory接入到Langchain中

使用场景是：读Memory -> Prompt -> Model -> Output -> 写Memory

#### (1) 不使用Memory的Chain

In [20]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

# 模型
llm = ChatOpenAI(model_name="gpt-4",api_key=openai.api_key ,base_url=openai.api_base)

# 提示词模版
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "您是一位乐于助人的AI小助手"),
        ("human", "{input}"),
    ]
)

# 链路
chat_chain = LLMChain(llm=llm, 
                         prompt=chat_template,
                         verbose=True)

In [21]:
chat_chain.invoke({"input":"你好，请你介绍一下你自己。"})



> Entering new LLMChain chain...
Prompt after formatting:
System: 您是一位乐于助人的AI小助手
Human: 你好，请你介绍一下你自己。

> Finished chain.


{'input': '你好，请你介绍一下你自己。',
 'text': '你好，很高兴见到你。我是一位虚拟的助手，我非常乐于助人。无论你有任何问题或者需要帮助，我都会尽我的能力为你提供信息和支持。我擅长处理各种问题，包括科技、学术、日常生活等各个领域，而且我是7*24小时在线的，随时准备提供帮助。希望我能成为你的可靠伙伴，和你一起探索和学习新的知识。'}

#### (2) 使用Memory的Chain

In [22]:
chat_chain = LLMChain(llm=llm, 
                      prompt=chat_template,
                      memory=InputStoreMemory(),   # 添加自定义的记忆类
                      verbose=True)

测试一下看看它是否在使用Memory

In [23]:
chat_chain.invoke({"input":"你好，请你介绍一下你自己。"})



> Entering new LLMChain chain...
Prompt after formatting:
System: 您是一位乐于助人的AI小助手
Human: 你好，请你介绍一下你自己。

> Finished chain.


{'input': '你好，请你介绍一下你自己。',
 'desc': '',
 'text': '你好！很高兴见到你。我是一位网络助手，乐于提供各种信息和建议。我可以帮助你调查问题，了解新闻，学习新技能，或者只是进行愉快的对话。我的知识面广泛且随时更新，所以无论你的问题是什么，我都会尽力提供最准确，最新的信息。希望你喜欢和我一起探索这个充满知识的世界！'}

In [24]:
chat_chain.invoke({"input":"我是小智"})



> Entering new LLMChain chain...
Prompt after formatting:
System: 您是一位乐于助人的AI小助手
Human: 我是小智

> Finished chain.


{'input': '我是小智',
 'desc': '你好，请你介绍一下你自己。',
 'text': '你好，小智！很高兴认识你。我是AI小助手，有什么可以帮助你的吗？'}

In [25]:
chat_chain.invoke({"input":"我现在每天都在服务我们的会员同学"})



> Entering new LLMChain chain...
Prompt after formatting:
System: 您是一位乐于助人的AI小助手
Human: 我现在每天都在服务我们的会员同学

> Finished chain.


{'input': '我现在每天都在服务我们的会员同学',
 'desc': '你好，请你介绍一下你自己。\n我是小智',
 'text': '哇，这听起来像是一份很有挑战性的任务！您能分享一下您在服务会员过程中遇到的一些有趣或者难忘的经历吗？'}

In [26]:
chat_chain.invoke({"input":"你知道小智每天都在做什么吗？"})



> Entering new LLMChain chain...
Prompt after formatting:
System: 您是一位乐于助人的AI小助手
Human: 你知道小智每天都在做什么吗？

> Finished chain.


{'input': '你知道小智每天都在做什么吗？',
 'desc': '你好，请你介绍一下你自己。\n我是小智\n我现在每天都在服务我们的会员同学',
 'text': '对不起，我并不能了解特定个体的日常活动，包括"小智"。我是一个AI助手，主要用于提供信息帮助和完成用户的请求，我无法获取或追踪个人的私人信息，除非用户主动向我提供。这是为了保护用户的隐私安全。'}

## 3. 自定义支持实体识别的Memory

之前的Memory一次性返回所有的聊天内容，比较草率。

我们希望实现一个Memory，它能够根据用户的提问，有选择地返回记忆内容。其中一个方法是借助实体识别(Entity Recognization)。

什么是Entity Recognization，举个例子，用户输入一段话，我们识别出这句话涉及了三个实体”马云”、“杭州”、“腾讯”。那么就用这三个实体来索引这三句话。在后续聊天中，当用户的提问涉及到“马云”或者“杭州”或者“腾讯”时，就可以从记忆中找到这段话

其实**Langchain也有**一个名为`ConversationEntityMemory`的类来提供**同样的功能**，本节我们手动实现一个来演示它的原理

### 3.1 spacy工具安装

接下来，我们做一个需求，演示一个稍复杂的`Memory`构建过程，该`Memory`使用 `spaCy`库提取实体并将有关它们的信息保存在一个简单的哈希表中。然后，在对话过程中，我们根据的输入文本，提取出输入文本中的实体，并将有关当前输入实体的记忆信息放入上下文中。

针对上述需求描述，我们补充两个知识点：首先，实体识别（Named Entity Recognition, NER）是自然语言处理（NLP）中的一个经典任务，其目的是从文本中识别出有特定意义的实体，例如人名、地点、组织机构名、时间表达式、数量、货币值等。实体识别通常作为信息提取、问答系统、内容摘要、语义搜索等应用的基础。其次，对于如何从一个文本中提取出具体的实体，我们要借助一个可以做实体识别的深度学习模型，该模型，我们选择从Python的一个`spacy`库中下载。

> spacy官网：https://spacy.io/

`spaCy`是一个开源的自然语言处理（NLP）库，提供了一些高性能的语言处理功能，适用于Python。主要用于文本分析和处理任务，包括但不限于词性标注、命名实体识别（NER）、句法依赖分析、句子边界检测等。使用`spaCy`库，需要在当前环境下安装其依赖包，执行如下代码：

In [28]:
! pip install --upgrade --quiet  spacy

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastapi-cli 0.0.2 requires typer>=0.12.3, but you have typer 0.9.4 which is incompatible.


In [ ]:
! python -m spacy download zh_core_web_sm

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
     |███████████████████████████     | 40.8 MB 50 kB/s eta 0:02:337